# Music Generation using an LSTM

#### Final Project for Deep Learning (CS 7643)

By Daeil Cha, Daniel Dias, Chitwan Kaudan

### Global Variables

In [ ]:
# data_path = "../../lmd_matched"
data_path = "../clean-data"

### Environment

In [ ]:
import torch

import numpy as np
import matplotlib.pyplot as plt

from IPython.core.debugger import set_trace
from getdata import getBatch

%matplotlib inline

%load_ext autoreload
%autoreload 2

batch_size = 16
learning_rate = 1e-6
epochs = 2

##### Pytorch GPU/CPU

In [ ]:
# dtype = torch.FloatTensor
# device = torch.device("cpu")

dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

### Model

In [ ]:
from model.main_model import MusicGeneration

model = MusicGeneration(time_sequence_len=64, time_hidden_size=36)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Data

In [ ]:
# how to get batches from getdata module
num_epochs = 10 # 1000
batch_size = 5
num_time_steps = 256
num_total_songs = 500
start = 0
batch = []

while start < num_total_songs:
    batch.append(np.array(getBatch(start, batch_size, num_time_steps, data_path)))
    # Shape should be (batch_size x num_time_steps x note_range x pitch/articulation)
    start += batch_size

In [ ]:
# data = load in data from folder
data = torch.randn(batch_size, 3)
y = torch.max(torch.randn(batch_size, 2), dim=1).indices

print(len(batch))

## Training

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(range(data_size)):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(data)

        # Compute and print loss
        # loss = criterion(torch.max(y_pred, dim=1).indices, y)
        loss = criterion(y_pred, y)
        if i % 100 == 99:
            print("Iteration:", i, "Loss:", loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

##### IPDB Test Code